# Prompt Shield Docs + Tutorial

This is a brief tutorial and documentation on using the Prompt Shield features of PyRIT.

## 1 Populating Environment Variables

The two environment variables you'll need are `AZURE_CONTENT_SAFETY_ENDPOINT` and `AZURE_CONTENT_SAFETY_KEY`. Make sure to add them to your .env file if you get an error.


In [2]:
from pyrit.common import default_values

default_values.load_default_env()

# 2 PromptShieldTarget

PromptShieldTarget is a target that uses Prompt Shield as its backend. Here's an example of how it processes input:

In [14]:
from pyrit.prompt_target import PromptShieldTarget
from pyrit.models import (
    PromptRequestPiece,
    PromptRequestResponse
)

import os
pst = PromptShieldTarget(
    os.environ.get('AZURE_CONTENT_SAFETY_ENDPOINT'),
    os.environ.get('AZURE_CONTENT_SAFETY_KEY')
)


In [15]:
prr = PromptRequestResponse(
    [
        PromptRequestPiece(
            role="user",
            original_value="hello",
            prompt_metadata="document" # the prompt_metadata specifies whether to use the document or the user classifier.
        )
    ]
)

In [16]:
response = await pst.send_prompt_async(prr)
print(response)

TypeError: PromptShieldTarget.send_prompt_async() takes 1 positional argument but 2 were given

In [1]:

from pyrit.score import PromptShieldScorer